<a href="https://colab.research.google.com/github/prhld0111/Mini-Project-3/blob/main/Cryptocurrency_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [ ]:
# Load the dataset
data = pd.read_csv("/content/sample_data/all_currencies.csv")

# Display the first few rows
print(data.head())


   Unnamed: 0        Date Symbol      Open      High       Low     Close  \
0           0  2015-11-12    $$$  0.000013  0.000017  0.000013  0.000017   
1           1  2015-11-13    $$$  0.000017  0.000033  0.000016  0.000024   
2           2  2015-11-14    $$$  0.000024  0.000053  0.000023  0.000030   
3           3  2015-11-15    $$$  0.000030  0.000063  0.000022  0.000035   
4           4  2015-11-16    $$$  0.000035  0.000046  0.000032  0.000036   

   Volume  Market Cap  
0   142.0         NaN  
1    85.0         NaN  
2   131.0         NaN  
3   132.0         NaN  
4   280.0         NaN  


In [ ]:
# Select relevant columns
data = data[['Date', 'Close', 'Open', 'High', 'Low', 'Volume', 'Market Cap']]

# Convert date to datetime format and sort
data['Date'] = pd.to_datetime(data['Date'])
data.sort_values('Date', inplace=True)

# Normalize numerical features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['Close', 'Open', 'High', 'Low', 'Volume', 'Market Cap']])

# Prepare the data for LSTM (use only 'close' for prediction)
data['scaled_close'] = scaled_data[:, 0]


In [ ]:
def create_sequences(data, look_back):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back, 0])  # Past prices
        y.append(data[i+look_back, 0])   # Future price
    return np.array(X), np.array(y)

look_back = 60  # Use 60 past days
X, y = create_sequences(scaled_data, look_back)

# Reshape X for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Initialize early stopping
early_stopping = EarlyStopping(
    monitor='val_loss',  # Metric to monitor
    patience=5,          # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

# Train the model with early stopping
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]  # Include early stopping in the callbacks list
)


Epoch 1/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 862s 53ms/step - loss: 1.2771e-05 - val_loss: 7.6025e-06
Epoch 2/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 927s 54ms/step - loss: 2.3538e-05 - val_loss: 7.0200e-06
Epoch 3/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 868s 54ms/step - loss: 1.7772e-05 - val_loss: 7.0183e-06
Epoch 4/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 919s 54ms/step - loss: 1.5928e-05 - val_loss: 7.2573e-06
Epoch 5/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 923s 54ms/step - loss: 1.6991e-05 - val_loss: 7.0438e-06
Epoch 6/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 865s 54ms/step - loss: 1.5376e-05 - val_loss: 7.0161e-06
Epoch 7/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 869s 54ms/step - loss: 1.5936e-05 - val_loss: 7.0150e-06
Epoch 8/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 919s 54ms/step - loss: 1.6818e-05 - val_loss: 7.0137e-06
Epoch 9/50
16144/16144 ━━━━━━━━━━━━━━━━━━━━ 925s 54ms/step - loss: 1.6494e-05 - val_loss: 7.0142e-06
Epoch 10/50
 6545/16144 ━━━━━━━━━━━━━━━━━━━━ 8:06 51ms/step - loss: 8.8641e-06

In [ ]:
# Plot training and validation loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.axvline(x=len(history.history['loss']) - early_stopping.patience, color='r', linestyle='--', label='Early Stopping Point')
plt.legend()
plt.show()
